In [10]:
import pandas as pd

# CSV 파일 불러오기
data = pd.read_csv("./bulkdata/add_emoji.csv")
print(data.columns)  # 컬럼 확인: description, tagged_description

# tagged_description 컬럼의 맨 앞 글자를 이모지로 간주하여 새로운 컬럼 생성
data['emoji'] = data['tagged_description'].str[0]

# 결과 확인
print(data[['description', 'tagged_description', 'emoji']].head())


Index(['description', 'tagged_description'], dtype='object')
                   description              tagged_description emoji
0  에어컨을 26도로 설정하고 필요한 시간에만 켠다.  ❄️ 에어컨을 26도로 설정하고 필요한 시간에만 켠다.     ❄
1      집 안의 모든 조명을 형광등으로 유지한다.       💡 집 안의 모든 조명을 형광등으로 유지한다.     💡
2   세탁물을 모아서 세탁기를 주 2~3회만 돌린다.     🧺세탁물을 모아서 세탁기를 주 2~3회만 돌린다.     🧺
3       전기밥솥 보온 기능을 하루 종일 켜둔다.        🔌 전기밥솥 보온 기능을 하루 종일 켜둔다.     🔌
4     사용하지 않는 전자제품의 플러그를 뽑아둔다.      🔌 사용하지 않는 전자제품의 플러그를 뽑아둔다.     🔌


In [12]:
# 이모지 라벨 인코딩 (유니크한 이모지에 번호 부여)
emoji_to_idx = {emoji: idx for idx, emoji in enumerate(data['emoji'].unique())}
idx_to_emoji = {idx: emoji for emoji, idx in emoji_to_idx.items()}

# 라벨 컬럼 추가
data['label'] = data['emoji'].apply(lambda x: emoji_to_idx[x])

print("라벨 인코딩 결과:", emoji_to_idx)


라벨 인코딩 결과: {'❄': 0, '💡': 1, '🧺': 2, '🔌': 3, '💧': 4, '🗑': 5, '🔥': 6, '🛢': 7}


In [13]:
from sentence_transformers import SentenceTransformer

# 사전 학습된 모델 선택
model_name = "paraphrase-MiniLM-L6-v2"
embedder = SentenceTransformer(model_name)

# description 컬럼에 대해 임베딩 계산 후 컬럼에 저장
data['embedding'] = data['description'].apply(lambda x: embedder.encode(x))

In [14]:
import torch
from torch.utils.data import Dataset, DataLoader

class EmojiDataset(Dataset):
    def __init__(self, df):
        self.embeddings = list(df['embedding'])
        self.labels = list(df['label'])
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        embedding = torch.tensor(self.embeddings[idx], dtype=torch.float)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return embedding, label

dataset = EmojiDataset(data)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [25]:
import torch.nn as nn

class EmojiClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(EmojiClassifier, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)
        
    def forward(self, x):
        return self.fc(x)

# 임베딩 벡터의 차원과 이모지 클래스 수 설정
input_dim = len(data['embedding'].iloc[0])
num_classes = len(emoji_to_idx)
classifier = EmojiClassifier(input_dim, num_classes)

# 학습 설정
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-3)
num_epochs = 1000  # 데이터 양에 따라 조정

# 학습 루프
avg_loss = 0
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for batch_embeddings, batch_labels in dataloader:
        outputs = classifier(batch_embeddings)
        loss = criterion(outputs, batch_labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(dataloader)
    # print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")
print(avg_loss)


0.11416801810264587


In [26]:
import pandas as pd

# 1. 새 CSV 파일 불러오기 (칼럼 이름이 "description"임을 확인)
val_data = pd.read_csv("./bulkdata/db-2025-04-01.csv")
print("불러온 데이터 미리보기:")
print(val_data.head())

# 2. 학습된 모델(예: classifier, embedder, idx_to_emoji)이 이미 존재한다고 가정하고, 
#    "description" 칼럼의 텍스트를 대상으로 예측 수행

def predict_emoji(sentence):
    with torch.no_grad():
        # 문장 임베딩 계산 (기존에 사용한 embedder)
        embedding = embedder.encode(sentence)
        embedding_tensor = torch.tensor(embedding, dtype=torch.float)
        # 모델 예측
        output = classifier(embedding_tensor)
        predicted_index = torch.argmax(output).item()
        return idx_to_emoji[predicted_index]

# 3. description 칼럼에 대해 예측 수행하여 새로운 컬럼에 저장
val_data['predicted_emoji'] = val_data['description'].apply(lambda x: predict_emoji(x))

# 4. 결과 출력
print("\n예측 결과 (description, predicted_emoji):")
print(val_data[['description', 'predicted_emoji']].head())

# 5. 원한다면 결과를 CSV 파일로 저장할 수 있습니다.
val_data.to_csv("./bulkdata/db_with_predictions.csv", index=False)



불러온 데이터 미리보기:
   eco_answer_id  eco_quiz_id                  description  exp  \
0              5          164  에어컨을 26도로 설정하고 필요한 시간에만 켠다.   20   
1              6          164      집 안의 모든 조명을 형광등으로 유지한다.  -15   
2              7          164  🧺세탁물을 모아서 세탁기를 주 2~3회만 돌린다.   12   
3              8          164       전기밥솥 보온 기능을 하루 종일 켜둔다.   -8   
4              9          164     사용하지 않는 전자제품의 플러그를 뽑아둔다.   10   

            created_at           updated_at  
0  2025-03-24 13:50:20  2025-03-24 13:50:20  
1  2025-03-24 13:50:20  2025-03-24 13:50:20  
2  2025-03-24 13:50:20  2025-03-31 10:58:28  
3  2025-03-24 13:50:20  2025-03-24 13:50:20  
4  2025-03-24 13:50:20  2025-03-24 13:50:20  

예측 결과 (description, predicted_emoji):
                   description predicted_emoji
0  에어컨을 26도로 설정하고 필요한 시간에만 켠다.               ❄
1      집 안의 모든 조명을 형광등으로 유지한다.               💡
2  🧺세탁물을 모아서 세탁기를 주 2~3회만 돌린다.               🧺
3       전기밥솥 보온 기능을 하루 종일 켜둔다.               🔌
4     사용하지 않는 전자제품의 플러그를 뽑아둔다.   

In [29]:
import pandas as pd

# 1. 정답 파일 불러오기
gt_data = pd.read_csv("./bulkdata/db_with_answer.csv")
print("정답 데이터 미리보기:")
print(gt_data.head())

# 2. description 칼럼을 대상으로 모델 예측 수행
# 이전에 정의한 predict_emoji 함수를 사용합니다.
gt_data['predicted_emoji'] = gt_data['description'].apply(lambda x: predict_emoji(x))

# 3. 예측 결과와 정답 비교
gt_data['correct'] = gt_data['predicted_emoji'] == gt_data['answer_emoji']

# 전체 정확도 계산
accuracy = gt_data['correct'].mean()
print(f"\n전체 정확도: {accuracy*100:.2f}%")

# 4. 오답 사례 출력 (원하는 경우)
mismatches = gt_data[~gt_data['correct']]
print("\n오답 사례 (description, answer_emoji, predicted_emoji):")
print(mismatches[['description', 'answer_emoji', 'predicted_emoji']].head())

# 5. 결과를 CSV 파일로 저장 (원하는 경우)
gt_data.to_csv("db_with_predictions_and_answer_comparison.csv", index=False)


정답 데이터 미리보기:
   eco_answer_id  eco_quiz_id                  description  exp  \
0              5          164  에어컨을 26도로 설정하고 필요한 시간에만 켠다.   20   
1              6          164      집 안의 모든 조명을 형광등으로 유지한다.  -15   
2              7          164  🧺세탁물을 모아서 세탁기를 주 2~3회만 돌린다.   12   
3              8          164       전기밥솥 보온 기능을 하루 종일 켜둔다.   -8   
4              9          164     사용하지 않는 전자제품의 플러그를 뽑아둔다.   10   

            created_at           updated_at answer_emoji  
0  2025-03-24 13:50:20  2025-03-24 13:50:20            ❄  
1  2025-03-24 13:50:20  2025-03-24 13:50:20            💡  
2  2025-03-24 13:50:20  2025-03-31 10:58:28            🧺  
3  2025-03-24 13:50:20  2025-03-24 13:50:20            🔌  
4  2025-03-24 13:50:20  2025-03-24 13:50:20            🔌  

전체 정확도: 41.05%

오답 사례 (descripttion, answer_emoji, predicted_emoji):
                      description answer_emoji predicted_emoji
5      음식물 찌꺼기를 물로 씻어 하수구에 흘려보낸다.            🥘               💧
7   기름 묻은 후라이팬을 키친타월로 닦은 후 설거지한다. 

In [31]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from sentence_transformers import SentenceTransformer

# 1. CSV 파일 불러오기
train_data = pd.read_csv("./bulkdata/db_with_answer.csv")
print("불러온 데이터 미리보기:")
print(train_data.head())

# 2. answer_emoji를 라벨로 사용
#    고유한 이모지에 번호를 부여하여 라벨 인코딩 진행
emoji_to_idx = {emoji: idx for idx, emoji in enumerate(train_data["answer_emoji"].unique())}
idx_to_emoji = {idx: emoji for emoji, idx in emoji_to_idx.items()}
train_data['label'] = train_data['answer_emoji'].apply(lambda x: emoji_to_idx[x])
print("라벨 인코딩 결과:", emoji_to_idx)

# 3. 임베딩 계산을 위한 사전학습 모델 불러오기
embedder = SentenceTransformer("paraphrase-MiniLM-L6-v2")

# 'description' 칼럼의 텍스트를 임베딩하고 새로운 칼럼에 저장
train_data['embedding'] = train_data['description'].apply(lambda x: embedder.encode(x))

# 4. PyTorch Dataset 정의
class EmojiDataset(Dataset):
    def __init__(self, df):
        self.embeddings = list(df['embedding'])
        self.labels = list(df['label'])
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        # 임베딩 벡터와 라벨을 텐서로 변환
        embedding = torch.tensor(self.embeddings[idx], dtype=torch.float)
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return embedding, label

dataset = EmojiDataset(train_data)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# 5. 분류기 모델 정의 (단순 선형 분류기)
class EmojiClassifier(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(EmojiClassifier, self).__init__()
        self.fc = nn.Linear(input_dim, num_classes)
        
    def forward(self, x):
        return self.fc(x)

input_dim = len(train_data['embedding'].iloc[0])  # 임베딩 벡터 차원
num_classes = len(emoji_to_idx)
classifier = EmojiClassifier(input_dim, num_classes)

# 6. 학습 설정
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(classifier.parameters(), lr=1e-3)
num_epochs = 1000  # 에포크 수는 데이터 양에 따라 조정

# 7. 재학습 (Training Loop)
for epoch in range(num_epochs):
    epoch_loss = 0.0
    for batch_embeddings, batch_labels in dataloader:
        outputs = classifier(batch_embeddings)
        loss = criterion(outputs, batch_labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        
    avg_loss = epoch_loss / len(dataloader)
    # print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")

# 재학습 완료 후, 모델을 저장하거나 추가 평가할 수 있습니다.


불러온 데이터 미리보기:
   eco_answer_id  eco_quiz_id                  description  exp  \
0              5          164  에어컨을 26도로 설정하고 필요한 시간에만 켠다.   20   
1              6          164      집 안의 모든 조명을 형광등으로 유지한다.  -15   
2              7          164  🧺세탁물을 모아서 세탁기를 주 2~3회만 돌린다.   12   
3              8          164       전기밥솥 보온 기능을 하루 종일 켜둔다.   -8   
4              9          164     사용하지 않는 전자제품의 플러그를 뽑아둔다.   10   

            created_at           updated_at answer_emoji  
0  2025-03-24 13:50:20  2025-03-24 13:50:20            ❄  
1  2025-03-24 13:50:20  2025-03-24 13:50:20            💡  
2  2025-03-24 13:50:20  2025-03-31 10:58:28            🧺  
3  2025-03-24 13:50:20  2025-03-24 13:50:20            🔌  
4  2025-03-24 13:50:20  2025-03-24 13:50:20            🔌  
라벨 인코딩 결과: {'❄': 0, '💡': 1, '🧺': 2, '🔌': 3, '🥘': 4, '💧': 5, '🗑': 6, '🛢': 7, '🔥': 8, '⚖️': 9}


In [32]:
import pandas as pd

# CSV 파일 불러오기
df = pd.read_csv("./bulkdata/db_with_answer.csv")

# answer_emoji와 descripttion을 결합하여 새로운 컬럼 생성
# (양쪽 사이에 공백 추가)
df['combined_text'] = df['answer_emoji'] + " " + df['description']

# 결과 미리보기
print(df[['answer_emoji', 'description', 'combined_text']].head())

# 새로운 CSV 파일로 저장 (원하는 경로 및 파일명으로 저장)
df.to_csv("./bulkdata/db_with_combined_text.csv", index=False)


  answer_emoji                  description                  combined_text
0            ❄  에어컨을 26도로 설정하고 필요한 시간에만 켠다.  ❄ 에어컨을 26도로 설정하고 필요한 시간에만 켠다.
1            💡      집 안의 모든 조명을 형광등으로 유지한다.      💡 집 안의 모든 조명을 형광등으로 유지한다.
2            🧺  🧺세탁물을 모아서 세탁기를 주 2~3회만 돌린다.  🧺 🧺세탁물을 모아서 세탁기를 주 2~3회만 돌린다.
3            🔌       전기밥솥 보온 기능을 하루 종일 켜둔다.       🔌 전기밥솥 보온 기능을 하루 종일 켜둔다.
4            🔌     사용하지 않는 전자제품의 플러그를 뽑아둔다.     🔌 사용하지 않는 전자제품의 플러그를 뽑아둔다.
